<a href="https://colab.research.google.com/github/spockthompson/Portfolio-Project/blob/main/Sales_Prediction_Project_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sales Predictions
- Soulution by: Scotty Thomason

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, \
OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

path = '/content/drive/MyDrive/sales_predictions.csv'
df = pd.read_csv(path)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


##Looking at the Data Set

In [2]:
df_ml = df.copy()

In [3]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
df_ml.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [5]:
df_ml.duplicated().sum()

0

In [6]:
df_ml.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

#Before splitting our data, We can drop duplicates and fix inconsistencies in categorical data.

In [7]:
# Fixing the description of items "Fat Content"
df_ml['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [8]:
df_ml['Item_Fat_Content'] = df_ml['Item_Fat_Content'].replace('LF', 'Low Fat')
df_ml['Item_Fat_Content'] = df_ml['Item_Fat_Content'].replace('reg','Regular')
df_ml['Item_Fat_Content'] = df_ml['Item_Fat_Content'].replace('low fat', 'Low Fat')

df_ml['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [9]:
# Ordinal Encoding 'Outlet_Size' , 'Outlet_Location_Type' and 'Outlet_Type'
replacement_dictionary = {'Small':0, 'Medium':1, 'High':2}
df_ml['Outlet_Size'].replace(replacement_dictionary, inplace=True)
df_ml['Outlet_Size']

0       1.0
1       1.0
2       1.0
3       NaN
4       2.0
       ... 
8518    2.0
8519    NaN
8520    0.0
8521    1.0
8522    0.0
Name: Outlet_Size, Length: 8523, dtype: float64

In [10]:
replacement_dictionary = {'Tier 1':0, 'Tier 2':1, 'Tier 3':2}
df_ml['Outlet_Location_Type'].replace(replacement_dictionary, inplace=True)
df_ml['Outlet_Location_Type']

0       0
1       2
2       0
3       2
4       2
       ..
8518    2
8519    1
8520    1
8521    2
8522    0
Name: Outlet_Location_Type, Length: 8523, dtype: int64

In [11]:
replacement_dictionary = {'Supermarket Type1':0, 'Supermarket Type2':1, 'Supermarket Type3':2, 'Grocery Store':3}
df_ml['Outlet_Type'].replace(replacement_dictionary, inplace=True)
df_ml['Outlet_Type']

0       0
1       1
2       0
3       3
4       0
       ..
8518    0
8519    0
8520    0
8521    1
8522    0
Name: Outlet_Type, Length: 8523, dtype: int64

#Identify the features (X) and target (y): Assign the "Item_Outlet_Sales" column as our target and the rest of the relevant variables as our features matrix and preform train test split.

In [12]:
# split X and y, you are predicting price
X = df_ml.drop(columns=['Item_Outlet_Sales', 'Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'])
y = df_ml['Item_Outlet_Sales']

# split training and test
# set random_state to 42 for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type
4776,16.350,Low Fat,0.029565,Household,256.4646,1.0,2,1
7510,15.250,Regular,0.000000,Snack Foods,179.7660,1.0,2,1
5828,12.350,Regular,0.158716,Meat,157.2946,1.0,0,0
5327,7.975,Low Fat,0.014628,Baking Goods,82.3250,0.0,1,0
4810,19.350,Low Fat,0.016645,Frozen Foods,120.9098,NaN,1,0


#Create a preprocessing object to prepare the dataset for Machine Learning

In [13]:
# Selectors
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')


In [14]:
# Imputers
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)


In [15]:
# Numeric pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

# Categorical pipeline
categorical_pipe = make_pipeline(freq_imputer, ohe)
categorical_pipe


Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [16]:
# Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple, remainder='drop')
preprocessor



ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa0339cdf10>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa0339cd250>)])

In [17]:
# fit on train
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa0339cdf10>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa0339cd250>)])

In [18]:
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# view results
# Check for missing values and that data is scaled and one-hot encoded
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')



0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (6392, 24)




## Creating a Linear Regression Model

In [19]:
# Create Model Pipeline
reg = LinearRegression()
reg_pipe = make_pipeline(preprocessor, reg)

In [20]:
# Model is learning the relationship between X and y
reg_pipe.fit(X_train,y_train)

train_pred = reg_pipe.predict(X_train)
test_pred = reg_pipe.predict(X_test)


In [21]:
# finding MAE, MSE, RMSE and R2 on the model for both the train and test data
def eval_regression(true, pred):
  mae = mean_absolute_error(true, pred)
  mse = mean_squared_error(true, pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(true, pred)

  print(f'RMSE: {rmse},\n R^2: {r2} ')

eval_regression(y_train, train_pred)

RMSE: 1338.8443827733095,
 R^2: 0.3943128533594993 


In [22]:
eval_regression(y_test, test_pred)

RMSE: 1296.5680721996916,
 R^2: 0.39068408671676114 


- The predictions are off quite a bit. Let's see if we can make this work better!

## Creating a Regression Tree Model

In [39]:
# Making instance of the model
dec_tree = DecisionTreeRegressor(random_state = 42)
dec_pipe = make_pipeline(preprocessor, dec_tree)

In [40]:
dec_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa0339cdf10>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa0339cd250>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [41]:
train_pred = dec_pipe.predict(X_train)
test_pred = dec_pipe.predict(X_test)

In [43]:
## Evaluate 
def evluate_regression(dec_tree,X_train,X_test,y_train,y_test):

    print(f"Training R^2: {dec_tree.score(X_train,y_train):.3f}")
    print(f"Test R^2: {dec_tree.score(X_test,y_test):.3f}")

evluate_regression(dec_pipe,X_train, X_test,y_train,y_test)

Training R^2: 1.000
Test R^2: 0.154


In [44]:
# What was the depth of our default tree?
dec_tree.get_depth()

42

In [46]:
# List of values to try for max_depth:
depths = list(range(1, 42))

dtscores = pd.DataFrame(index=depths, columns=['Test Score','Train Score'])
for depth in depths:
    dec_tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    dec_tree.fit(X_train_processed, y_train)
    train_score = dec_tree.score(X_train_processed, y_train)
    test_score = dec_tree.score(X_test_processed, y_test)
    dtscores.loc[depth, 'Train Score'] = train_score
    dtscores.loc[depth, 'Test Score'] = test_score


In [47]:
sorted_scores = dtscores.sort_values(by='Test Score', ascending=False)
sorted_scores.head()

,Test Score,Train Score
5,0.59471,0.60394
4,0.584005,0.582625
6,0.582587,0.615072
7,0.578866,0.626453
8,0.565395,0.642791


In [48]:
# Let's run the model with our optimized value for max_depth
opt_dec = DecisionTreeRegressor(max_depth = 5, random_state = 42)
opt_dec.fit(X_train_processed, y_train)

train_5_score = opt_dec.score(X_train_processed, y_train)
test_5_score = opt_dec.score(X_test_processed, y_test)
print(train_5_score)
print(test_5_score)

0.6039397477322956
0.5947099753159972


In [49]:
eval_regression(y_train, train_pred)

RMSE: 5.50728349323243e-15,
 R^2: 1.0 


In [50]:
eval_regression(y_test, test_pred)

RMSE: 1527.3841037820885,
 R^2: 0.1544322561854664 
